In [1]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from datetime import datetime 

import time

In [2]:
#  parameters
circle="Zone"
month="nov2023"
month_doc=9
month_unpaid=12
date='13122023'
ftpHost = 'ftp.uppclonline.com'
ftpUname = 'mvftpreport'
ftpPass = 'Mvftp@321'
localFolderPath = f".\\{month}\\billed"
remoteFolder = f"CSV_BILLED/{date}"

In [3]:
def NonGOVT(df):
    convert_dict = {'SUPPLY_TYPE': str}
    df = df.astype(convert_dict)
    df['SUPPLY_TYPE'] = df['SUPPLY_TYPE'].astype(str).str.replace('\\.0+$', '')
    govt = ['30','31','32','33','34','35','36','37','38','39','40','40A', '41', '41A', '42', '42A', '43', '44', '45','48','72','73','74','75','80','81','82','83','100','101','102','103','104','105','106','107','80B','42A','41A','40A','H31'
           'H32','H33','H41','H42','H43','HV41','H13','H14']
    govt = [str(x) for x in govt]
    df['gov_flag'] = df['SUPPLY_TYPE'].isin(govt)
    return df

In [4]:
def categories(df):
    #Supplytype.srs.startswith('1') for lmv1
    convert_dict = {'SUPPLY_TYPE': str,
                
                }
 
    df = df.astype(convert_dict)
    df['SUPPLY_TYPE'] = df['SUPPLY_TYPE'].astype(str).str.replace('\\.0+$', '')
    df['Cat'] = None
#     df[df['SUPPLY_TYPE'].str.startswith('1')] = "LMV1"
    df.loc[df['SUPPLY_TYPE'].str.startswith('1', na=False),"Cat"] = "LMV1"
    df.loc[df['SUPPLY_TYPE'].str.startswith('2',na=False),"Cat"] = "LMV2"
    df.loc[df['SUPPLY_TYPE'].str.startswith('3', na=False),"Cat"] = "LMV3"
    # df.loc[df['SUPPLY_TYPE'].str.startswith('4', na=False),"Cat"] = "LMV4"
    df.loc[df['SUPPLY_TYPE'].str.startswith('5', na=False),"Cat"] = "LMV5"
    df.loc[df['SUPPLY_TYPE'].str.startswith('6', na=False),"Cat"] = "LMV6"
    df.loc[df['SUPPLY_TYPE'].str.startswith('7', na=False),"Cat"] = "LMV7"
    df.loc[df['SUPPLY_TYPE'].str.startswith('8', na=False),"Cat"] = "LMV8"
    df.loc[df['SUPPLY_TYPE'].str.startswith('9', na=False),"Cat"] = "LMV9"
#     df.loc[df['SUPPLY_TYPE'].str.startswith('10'),"Cat"] = "LMV10"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H1', na=False),"Cat"] = "HV1"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H2', na=False),"Cat"] = "HV2"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H3', na=False),"Cat"] = "HV3"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H4', na=False),"Cat"] = "HV4"
    lmv10 = ['100', '101', '103', '104', '105']
    # df['gov_flag'] = df['SUPPLY_TYPE'].isin(lmv10)
    df.loc[df['SUPPLY_TYPE'].isin(lmv10),"Cat"] = "LMV10"
    lmv4a = ['40','40A', '41', '41A', '42', '42A', ]
    lmv4b = ['46', '47','43', '44', '45','48']
    df.loc[df['SUPPLY_TYPE'].isin(lmv4a),"Cat"] = "LMV4A"
    df.loc[df['SUPPLY_TYPE'].isin(lmv4b),"Cat"] = "LMV4B"
    return df

In [5]:
def CatLoad(df):
    df['SANCTION_LOAD'] = pd.to_numeric(df['SANCTION_LOAD'])   #,errors="coerce"
    df['LOAD_FILTER'] = None
    df.loc[df['SANCTION_LOAD'] < 5,"LOAD_FILTER"] = "1-5kw"
    df.loc[((df['SANCTION_LOAD'] >= 5) & (df['SANCTION_LOAD'] < 10)),"LOAD_FILTER"] = "5-9kw"
    df.loc[df['SANCTION_LOAD'] >= 10,"LOAD_FILTER"] = "great than 10kw"
    return df


In [6]:
# def billable(df):
#     bil = ['In Service','TD']
#     df['bilable_flag'] = df['CON_STATUS'].isin(bil)
#     return df
  

In [7]:
import os, shutil
folder = f".\\{month}\\billed"
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))
        
folder = f".\\{month}\\unbilled"
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))


In [8]:
# df_master = pd.read_csv(f".\\{month}\\{circle}\\Zonesep23_8oct.csv")

In [9]:
# df_master.columns

In [10]:
# df_master = df_master[df_master['Circle_Name'] == 'EDC Bahraich']

In [11]:
# # quit and close the connection
# import ftplib


# def getFtpFilenames(ftpHost,  ftpUname, ftpPass, remoteWorkingDirectory):
#     # create an FTP client instance, use the timeout(seconds) parameter for slow connections only
#     ftp = ftplib.FTP(timeout=30)
    
#     # connect to the FTP server
#     ftp.connect(ftpHost)
    
#     # login to the FTP server
#     ftp.login(ftpUname, ftpPass)

#     # change current working directory if specified
#     if not (remoteWorkingDirectory == None or remoteWorkingDirectory.strip() == ""):
#         _ = ftp.cwd(remoteWorkingDirectory)
    
#     # initialize the filenames as an empty list
#     fnames = []
    
#     try:
#         # use nlst function to get the list of filenames
#         fnames = ftp.nlst()
#     except ftplib.error_perm as resp:
#         if str(resp) == "550 No files found":
#             fnames = []
#         else:
#             raise
    
#     # send QUIT command to the FTP server and close the connection
#     ftp.quit()

#     # return the list of filenames
#     return fnames

# # connection parameters
# ftpHost = 'ftp.uppclonline.com'
# ftpUname = 'mvftpreport'
# ftpPass = 'Mvftp@321'
# ftpcwd = f'CSV_BILLED/{date}/'
# fnames = getFtpFilenames(ftpHost,ftpUname, ftpPass,ftpcwd )
# print(fnames)
# print("execution complete...")

In [12]:
def low_consumtion_filter(df):
    
#     df = df[df['TOTAL_OUTSTANDING'] > 0]
    df['low_consumtion_filter'] = None
    df.loc[df['BILLED_UNITS'] <= 10,"low_consumtion_filter"] = "0-10"
    df.loc[((df['BILLED_UNITS'] > 10 ) & (df['BILLED_UNITS'] <= 30)),"low_consumtion_filter"] = "10-30"
    df.loc[((df['BILLED_UNITS'] > 30) & (df['BILLED_UNITS'] <= 50)),"low_consumtion_filter"] = "30-50"
    df.loc[((df['BILLED_UNITS'] > 50) & (df['BILLED_UNITS'] <=100)),"low_consumtion_filter"] = "50-100"
    df.loc[df['BILLED_UNITS'] > 100,"low_consumtion_filter"] = "> 100"
    
    return df

In [13]:
def unpaid(df):
    df['paid_status_curr'] = 0
    # df.loc[df['LAST_PAY_DATE'] < pd.Timestamp(2023, 1, month_unpaid),"unpaid_status"] = 0
    # df.loc[(pd.isnull(df['LAST_PAY_DATE'] )),"unpaid_status"] = 0     
    df.loc[df['LAST_PAY_DATE'] >= pd.Timestamp(2023, month_unpaid,1),"paid_status_curr"] = 1
    
    return df

In [14]:
def curr_paid_amount(df):
    df['curr_paid_amount'] = 0
    df.loc[df['paid_status_curr'] == 1,"curr_paid_amount"] = df[df['paid_status_curr'] == 1]['LAST_PAYMENT_AMOUNT']
    
    return df
    

In [15]:
def correction():
    # rankings_pd.rename(columns = {'test':'TEST'}, inplace = True)
    unbilled.rename(columns = {'LAST_BILL_DATE':'BILL_DATE','LAST_PAY_AMT':'LAST_PAYMENT_AMOUNT'}, inplace = True)
    unbilled.drop(['KNO', 'BOOK_NO', 'DISCOM', 'ZONE_CODE','CIRCLE_CODE','DIV_CODE', 'SDO_CODE','SCNO','MTR_SRL_NO','METER_STATUS','ACCT_INF_FLG'], axis=1, inplace=True)
    billed.drop(['BOOK_NO', 'SCNO', 'DISCOM', 'ZONE_CODE', 'CIRCLE_CODE','SDO_CODE'], axis=1, inplace=True)
    return

In [16]:
import ftplib
import os
import gzip
import shutil
import time



def downloadFilesFromFtp(localfolderPath, targetFilenames, ftpHost, ftpUname, ftpPass, remoteWorkingDirectory):
    # initialize the flag that specifies if download is success
    isDownloadSuccess: bool = False

    # create an FTP client instance, use the timeout parameter for slow connections only
    ftp = ftplib.FTP(timeout=30)
    ftp.encoding = "utf-8"

    # connect to the FTP server
    ftp.connect(ftpHost)

    # login to the FTP server
    ftp.login(ftpUname, ftpPass)

    # change current working directory if specified
    if not (remoteWorkingDirectory == None or remoteWorkingDirectory.strip() == ""):
        _ = ftp.cwd(remoteWorkingDirectory)

    # iterate through each remote filename and download
    for fItr in range(len(targetFilenames)):
        targetFilename = targetFilenames[fItr]
        print(targetFilename)
        # derive the local file path by appending the local folder path with remote filename
        localFilePath = os.path.join(localfolderPath, targetFilename)
        print(localFilePath)
        print("downloading file {0}".format(targetFilename))
        # download FTP file using retrbinary function
        with open(localFilePath, "wb") as file:
            retCode = ftp.retrbinary("RETR " + targetFilename, file.write)
         
        with gzip.open(localFilePath, 'rb') as f_in:
            with open(localFilePath[:-3], 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        os.remove(localFilePath)
    # read file
            
    # send QUIT command to the FTP server and close the connection
    ftp.quit()

    # check if download is success using the return code (retCode)
    if retCode.startswith('226'):
        isDownloadSuccess = True
    return isDownloadSuccess



remoteFilenames = [ f'BILLED_MVVNL_DIV354921_{date}.csv.gz', f'BILLED_MVVNL_DIV354922_{date}.csv.gz', f'BILLED_MVVNL_DIV354923_{date}.csv.gz', f'BILLED_MVVNL_DIV355131_{date}.csv.gz', f'BILLED_MVVNL_DIV355132_{date}.csv.gz', f'BILLED_MVVNL_DIV355133_{date}.csv.gz', f'BILLED_MVVNL_DIV355211_{date}.csv.gz', f'BILLED_MVVNL_DIV355212_{date}.csv.gz', f'BILLED_MVVNL_DIV355213_{date}.csv.gz', f'BILLED_MVVNL_DIV355214_{date}.csv.gz']


# run the function to download the files from FTP server
isDownloadSuccess = downloadFilesFromFtp(
    localFolderPath,remoteFilenames, ftpHost,ftpUname, ftpPass, remoteFolder)

print("download status = {0}".format(isDownloadSuccess))

BILLED_MVVNL_DIV354921_13122023.csv.gz
.\nov2023\billed\BILLED_MVVNL_DIV354921_13122023.csv.gz
downloading file BILLED_MVVNL_DIV354921_13122023.csv.gz
BILLED_MVVNL_DIV354922_13122023.csv.gz
.\nov2023\billed\BILLED_MVVNL_DIV354922_13122023.csv.gz
downloading file BILLED_MVVNL_DIV354922_13122023.csv.gz


TimeoutError: timed out

In [ ]:
localFolderPath = f".\\{month}\\unbilled"
remoteFolder = f"CSV_UNBILLED/{date}"

In [ ]:
import ftplib
import os
import gzip
import shutil
import time



def downloadFilesFromFtp(localfolderPath, targetFilenames, ftpHost, ftpUname, ftpPass, remoteWorkingDirectory):
    # initialize the flag that specifies if download is success
    isDownloadSuccess: bool = False

    # create an FTP client instance, use the timeout parameter for slow connections only
    ftp = ftplib.FTP(timeout=30)
    ftp.encoding = "utf-8"

    # connect to the FTP server
    ftp.connect(ftpHost)

    # login to the FTP server
    ftp.login(ftpUname, ftpPass)

    # change current working directory if specified
    if not (remoteWorkingDirectory == None or remoteWorkingDirectory.strip() == ""):
        _ = ftp.cwd(remoteWorkingDirectory)

    # iterate through each remote filename and download
    for fItr in range(len(targetFilenames)):
        targetFilename = targetFilenames[fItr]
        print(targetFilename)
        # derive the local file path by appending the local folder path with remote filename
        localFilePath = os.path.join(localfolderPath, targetFilename)
        print(localFilePath)
        print("downloading file {0}".format(targetFilename))
        # download FTP file using retrbinary function
        with open(localFilePath, "wb") as file:
            retCode = ftp.retrbinary("RETR " + targetFilename, file.write)
         
        with gzip.open(localFilePath, 'rb') as f_in:
            with open(localFilePath[:-3], 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        os.remove(localFilePath)
    # read file
            
    # send QUIT command to the FTP server and close the connection
    ftp.quit()

    # check if download is success using the return code (retCode)
    if retCode.startswith('226'):
        isDownloadSuccess = True
    return isDownloadSuccess


remoteFilenames = [ f'UNBILLED_MVVNL_DIV354921_{date}.csv.gz', f'UNBILLED_MVVNL_DIV354922_{date}.csv.gz', f'UNBILLED_MVVNL_DIV354923_{date}.csv.gz', f'UNBILLED_MVVNL_DIV355131_{date}.csv.gz', f'UNBILLED_MVVNL_DIV355132_{date}.csv.gz', f'UNBILLED_MVVNL_DIV355133_{date}.csv.gz', f'UNBILLED_MVVNL_DIV355211_{date}.csv.gz', f'UNBILLED_MVVNL_DIV355212_{date}.csv.gz', f'UNBILLED_MVVNL_DIV355213_{date}.csv.gz', f'UNBILLED_MVVNL_DIV355214_{date}.csv.gz']

# run the function to download the files from FTP server
isDownloadSuccess = downloadFilesFromFtp(
    localFolderPath,remoteFilenames, ftpHost,ftpUname, ftpPass, remoteFolder)

print("download status = {0}".format(isDownloadSuccess))

In [ ]:
bill="billed"

In [ ]:
def loop(file_name):
    return pd.read_csv(f"{month}/{bill}/{file_name}")

In [ ]:
mylist = os.listdir(f"{month}/{bill}")

In [ ]:
mylist

In [ ]:
start = time.time()
billed = []
billed = Parallel(n_jobs=-1, verbose=10,prefer="threads")(delayed(loop)(file_name) for file_name in mylist)
billed = pd.concat(billed, ignore_index=True)

end = time.time()
print("CSV:", end - start)

In [ ]:
bill="unbilled"
def loop(file_name):
    return pd.read_csv(f"{month}/{bill}/{file_name}")
mylist = os.listdir(f"{month}/{bill}")
mylist

In [ ]:
start = time.time()
unbilled = []
unbilled = Parallel(n_jobs=-1, verbose=10,prefer="threads")(delayed(loop)(file_name) for file_name in mylist)
unbilled = pd.concat(unbilled, ignore_index=True)

end = time.time()
print("CSV:", end - start)

In [ ]:


billed['bill'] = 1
unbilled['bill'] = 0
correction()
data=[]
data = [billed,unbilled]

In [ ]:
billed.columns

In [ ]:
unbilled.columns

In [ ]:
# bill_unbill = pd.concat(data, ignore_index=True)

In [ ]:
billed = billed[billed['ACCT_ID'].notna()]

In [ ]:
# bill_unbill.PAYMENT_MODE.value_counts()

In [ ]:
# unbilled['LAST_PAY_DATE'] = pd.to_datetime(unbilled['LAST_PAY_DATE'],dayfirst=True,errors ='coerce')

In [ ]:
# unbilled = unpaid(unbilled)

In [ ]:
# billed[billed['paid_status_curr']==1].to_csv(f"{month}/{circle}/{circle}_{date}_billed.csv") 

In [ ]:
# unbilled[unbilled['paid_status_curr']==1].to_csv(f"{month}/{circle}/{circle}_{date}_unbilled.csv") 

In [ ]:

BILLED_PAY=(billed.pivot_table(index='DIV_NAME', columns=['TARIFF_TYPE'],values=['LAST_PAYMENT_AMOUNT'],
                    aggfunc={'LAST_PAYMENT_AMOUNT':np.sum},fill_value=0,margins=True,margins_name='Total' ))


In [ ]:

UNBILLED_PAY=(unbilled.pivot_table(index='DIV_NAME', columns=['TARIFF_TYPE'],values=['LAST_PAY_AMT'],
                    aggfunc={'LAST_PAY_AMT':np.sum},fill_value=0,margins=True,margins_name='Total' ))


In [ ]:
UNBILLED_PAY.to_excel(f"{month}/{circle}/{circle}_UNBILLED_PAY.xlsx")



In [ ]:
bill_unbill = pd.concat(data, ignore_index=True)

In [ ]:
bill_unbill.columns

In [ ]:
bill_unbill['ACCT_ID'] = pd.to_numeric(bill_unbill['ACCT_ID'],errors='coerce')
bill_unbill.dropna(subset = ['ACCT_ID'], inplace=True)
bill_unbill['BILLED_UNITS'] = pd.to_numeric(bill_unbill['BILLED_UNITS'],errors='coerce')
bill_unbill['LAST_PAY_DATE'] = pd.to_datetime(billed['LAST_PAY_DATE'],dayfirst=True,errors='coerce')
bill_unbill['BILL_DATE'] = pd.to_datetime(bill_unbill['BILL_DATE'],dayfirst=True,errors='coerce')

In [ ]:
bill_unbill['LAST_PAYMENT_AMOUNT'] = pd.to_numeric(bill_unbill['LAST_PAYMENT_AMOUNT'],errors='coerce')

In [ ]:
# bill_unbill['LAST_PAY_DATE'] = pd.to_datetime(billed['LAST_PAY_DATE'],dayfirst=True,errors ='coerce')

bill_unbill['PAYMENT_MODE']=bill_unbill['PAYMENT_MODE'].str.strip()

In [ ]:
bill_unbill['PAYMENT_MODE'].value_counts()

In [ ]:
bill_unbill['DIV_NAME'].shape

In [ ]:
column =list(billed.columns)

In [ ]:
column

In [ ]:
imp_column =['ACCT_ID','MOBILE_NO',
 'TARIFF_TYPE',
             'SUPPLY_TYPE',
 'SANCTION_LOAD',
 'SANCTION_LOAD_UOM',
             'MF',
 'SUBSTATION',
 'FEEDER','BILL_BASIS', 'BILL_DATE',
 'BILLED_UNITS',

 'LAST_PAY_DATE',
 'LAST_PAYMENT_AMOUNT',
 'AMOUNT_PAYABLE',

 'CA',
 'PAYMENT_MODE'
             
            ]

In [ ]:
billed.info()

In [ ]:
billed.BILL_BASIS.value_counts()

In [ ]:
billed.TARIFF_TYPE.value_counts()

In [ ]:
bill_unbill = low_consumtion_filter(bill_unbill)
bill_unbill = unpaid(bill_unbill)
bill_unbill = curr_paid_amount(bill_unbill)
# bill_unbill = billable(bill_unbill)
bill_unbill = NonGOVT(bill_unbill)


In [ ]:
bill_unbill = CatLoad(bill_unbill)

In [ ]:
bill_unbill.columns

In [ ]:
# df1=bill_unbill[bill_unbill['LOAD_FILTER'] == "great than 10kw"]

In [ ]:
bill_unbill = categories(bill_unbill)

In [ ]:
bill_unbill['Cat'].value_counts()

In [ ]:
cat = bill_unbill['Cat'].unique() 
cat

In [ ]:
cat = [ 'LMV2', 'LMV6','LMV4']

In [ ]:
div_billed=billed[(billed.TARIFF_TYPE.isin(cat))].pivot_table(index=['CIRCLE_NAME','DIV_NAME'], columns=['BILL_BASIS'],values=['ACCT_ID'],
                    aggfunc={'ACCT_ID':'count'},fill_value=0 )

In [ ]:
div_billed.to_excel(f"{month}/zone/report/{circle}_{date}_billed.xlsx")

In [ ]:
div_csc_other_nov=bill_unbill[((bill_unbill.paid_status_curr==1) & (bill_unbill.PAYMENT_MODE=='EGSIL001'))].pivot_table(index=['LAST_PAY_DATE'], columns=['CIRCLE_NAME','DIV_NAME'],values=['LAST_PAYMENT_AMOUNT'],
                    aggfunc={'LAST_PAYMENT_AMOUNT':np.sum},fill_value=0 )

In [ ]:
div_payment_nov=bill_unbill[((bill_unbill.paid_status_curr==1) & (bill_unbill.TARIFF_TYPE!='LMV3')&(bill_unbill.TARIFF_TYPE!='LMV7'))].pivot_table(index=['LAST_PAY_DATE'], columns=['CIRCLE_NAME','DIV_NAME'],values=['LAST_PAYMENT_AMOUNT'],
                    aggfunc={'LAST_PAYMENT_AMOUNT':np.sum},fill_value=0 )

In [ ]:
div_payment_all_nov = bill_unbill[((bill_unbill.paid_status_curr == 1)&(bill_unbill.Cat.isin(cat)))].pivot_table(
    index=['LAST_PAY_DATE'], columns=['CIRCLE_NAME', 'DIV_NAME'], values=['LAST_PAYMENT_AMOUNT'],
    aggfunc={'LAST_PAYMENT_AMOUNT': np.sum}, fill_value=0
)

In [ ]:
div_payment_37_nov = bill_unbill[((bill_unbill.paid_status_curr == 1) & ((bill_unbill.TARIFF_TYPE == 'LMV3') | (bill_unbill.TARIFF_TYPE == 'LMV7')))].pivot_table(
    index=['LAST_PAY_DATE'], columns=['CIRCLE_NAME', 'DIV_NAME'], values=['LAST_PAYMENT_AMOUNT'],
    aggfunc={'LAST_PAYMENT_AMOUNT': np.sum}, fill_value=0
)

In [ ]:
zone_merge_28['TARIFF_TYPE'].value_counts()

In [ ]:
div_load_nov=bill_unbill[((bill_unbill.TARIFF_TYPE!='LMV5')& (bill_unbill.gov_flag== False))].pivot_table(index=['CIRCLE_NAME','DIV_NAME'], columns=['LOAD_FILTER'],values=['ACCT_ID','bill','paid_status_curr'],
                    aggfunc={'ACCT_ID':'count','bill':np.sum,'paid_status_curr':np.sum,},fill_value=0 )

In [ ]:
div_csc_other_nov.to_excel(f"{month}/{circle}/report/{circle}_{date}_CSC.xlsx")

In [ ]:
df1.to_excel(f"{month}/{circle}_{date}_10.xlsx")

In [ ]:
div_load_nov.to_excel(f"{month}/{circle}/report/{circle}_{date}_load.xlsx")

In [ ]:
div_payment_37_nov.to_excel(f"{month}/{circle}/report/{circle}_{date}_payment37.xlsx")

In [ ]:
div_payment_all_nov.to_excel(f"{month}/{circle}/report/{circle}_{date}_paymentnot.xlsx")

In [ ]:
zone_merge_28.to_csv(f"{month}/{circle}/{circle}_{date}_merge.csv") 